In [95]:
from mlagents_envs.registry import default_registry

environment_names = list(default_registry.keys())
for name in environment_names:
   print(name)

Basic
3DBall
3DBallHard
GridWorld
Hallway
VisualHallway
CrawlerDynamicTarget
CrawlerStaticTarget
Bouncer
SoccerTwos
PushBlock
VisualPushBlock
WallJump
Tennis
Reacher
Pyramids
VisualPyramids
Walker
FoodCollector
VisualFoodCollector
StrikersVsGoalie
WormStaticTarget
WormDynamicTarget


In [96]:
env = default_registry["3DBall"].make()
env.reset()

Found path: /tmp/ml-agents-binaries/binaries/3DBall-b1b94a0ae13eef9d91f9d8db1e5770c5/Startup/Startup.x86_64


## Get the Behavior Specs from the Environment

In [97]:
# We will only consider the first Behavior
behavior_name = list(env.behavior_specs)[0]
print(f"Name of the behavior : {behavior_name}")
spec = env.behavior_specs[behavior_name]

Name of the behavior : 3DBall?team=0


## Get the Observation Space from the Behavior Specs

In [98]:
# Examine the number of observations per Agent
print("Number of observations : ", len(spec.observation_specs))

# Is there a visual observation ?
# Visual observation have 3 dimensions: Height, Width and number of channels
vis_obs = any(len(spec.shape) == 3 for spec in spec.observation_specs)
print("Is there a visual observation ?", vis_obs)

Number of observations :  1
Is there a visual observation ? False


## Get the Action Space from the Behavior Specs

In [99]:
# Is the Action continuous or multi-discrete ?
if spec.action_spec.continuous_size > 0:
  print(f"There are {spec.action_spec.continuous_size} continuous actions")
if spec.action_spec.is_discrete():
  print(f"There are {spec.action_spec.discrete_size} discrete actions")


# How many actions are possible ?
#print(f"There are {spec.action_size} action(s)")

# For discrete actions only : How many different options does each action has ?
if spec.action_spec.discrete_size > 0:
  for action, branch_size in enumerate(spec.action_spec.discrete_branches):
    print(f"Action number {action} has {branch_size} different options")


There are 2 continuous actions


Get the steps from the Environment

You can do this with the env.get_steps(behavior_name) method. If there are multiple behaviors in the Environment, you can call this method with each of the behavior's names. Note This will not move the simulation forward.

In [100]:
decision_steps, terminal_steps = env.get_steps(behavior_name)

#### Set actions for each behavior
You can set the actions for the Agents of a Behavior by calling `env.set_actions()` you will need to specify the behavior name and pass a tensor of dimension 2. The first dimension of the action must be equal to the number of Agents that requested a decision during the step.

In [101]:
env.set_actions(behavior_name, spec.action_spec.empty_action(len(decision_steps)))

#### Move the simulation forward
Call `env.step()` to move the simulation forward. The simulation will progress until an Agent requestes a decision or terminates.

In [102]:
env.step()

In [103]:
len(decision_steps)

12

In [104]:
print(spec.action_spec.empty_action(len(decision_steps)))

#### Show the observations for one of the Agents
`DecisionSteps.obs` is a tuple containing all of the observations for all of the Agents with the provided Behavior name.
Each value in the tuple is an observation tensor containing the observation data for all of the agents.

In [105]:
import matplotlib.pyplot as plt
%matplotlib inline

for index, obs_spec in enumerate(spec.observation_specs):
  if len(obs_spec.shape) == 3:
    print("Here is the first visual observation")
    plt.imshow(decision_steps.obs[index][0,:,:,:])
    plt.show()

for index, obs_spec in enumerate(spec.observation_specs):
  if len(obs_spec.shape) == 1:
    print("First vector observations : ", decision_steps.obs[index][0,:])

First vector observations :  [-0.01467304 -0.01468306 -0.5208206   4.         -0.79952097  0.
  0.          0.        ]


### Run the Environment for a few episodes

In [106]:
for episode in range(3):
  env.reset()
  decision_steps, terminal_steps = env.get_steps(behavior_name)
  tracked_agent = -1 # -1 indicates not yet tracking
  done = False # For the tracked_agent
  episode_rewards = 0 # For the tracked_agent
  while not done:
    # Track the first agent we see if not tracking
    # Note : len(decision_steps) = [number of agents that requested a decision]
    if tracked_agent == -1 and len(decision_steps) >= 1:
      tracked_agent = decision_steps.agent_id[0]

    # Generate an action for all agents
    action = spec.action_spec.random_action(len(decision_steps))
    print(action.continuous)

    # Set the actions
    env.set_actions(behavior_name, action)

    # Move the simulation forward
    env.step()

    # Get the new simulation results
    decision_steps, terminal_steps = env.get_steps(behavior_name)
    if tracked_agent in decision_steps: # The agent requested a decision
      episode_rewards += decision_steps[tracked_agent].reward
    if tracked_agent in terminal_steps: # The agent terminated its episode
      episode_rewards += terminal_steps[tracked_agent].reward
      done = True
  print(f"Total rewards for episode {episode} is {episode_rewards}")

[[ 0.18055348 -0.59612787]
 [-0.8160639   0.9320135 ]
 [ 0.36734298  0.9799873 ]
 [-0.27983254  0.14790481]
 [ 0.28260988  0.6531011 ]
 [-0.8358733  -0.2810583 ]
 [ 0.26461753 -0.4614528 ]
 [ 0.65129215  0.05826994]
 [ 0.97804546 -0.9602319 ]
 [ 0.8566099   0.455045  ]
 [-0.6925044   0.7030451 ]
 [ 0.09986396 -0.32880056]]
[[ 0.09541556  0.04835454]
 [-0.93291044  0.27761763]
 [ 0.52699023 -0.8071744 ]
 [-0.23612864 -0.04265133]
 [ 0.6790372  -0.99544615]
 [-0.8794737   0.06079737]
 [ 0.8818696   0.9433242 ]
 [ 0.8838776  -0.6799025 ]
 [ 0.9834978   0.20562449]
 [-0.1572911   0.51278585]
 [ 0.06421845  0.39980674]
 [-0.4673556   0.43255305]]
[[ 0.7116972   0.6247719 ]
 [ 0.9400138  -0.7904913 ]
 [ 0.6319336   0.25219876]
 [ 0.79683375  0.6237456 ]
 [ 0.17644836 -0.48040965]
 [ 0.5945343  -0.08471713]
 [-0.48946816 -0.5869842 ]
 [ 0.90868664  0.16358082]
 [-0.3614937  -0.72899103]
 [ 0.43256903  0.41441974]
 [-0.06185324  0.09068146]
 [ 0.6445432   0.4765489 ]]
[[-0.9516511   0.6610703 

In [107]:
env.close()
print("Closed environment")

Closed environment
